# Übung 4.1 - Neuronale Netze am Beispiel Presshärten
VO Maschinelles Lernen in der Produktion, WS2020/21, Moritz von Unold, Richard Lux, Felix Soest

#### In diesem Notebook wird das Verfahren Neuronale Netze anhand des Anwendungsbeispiels Rattern geübt.

Ziel des trainierten Neuronalen Netzes ist hier den Zusammenhang zwischen den 7 Prozess-Parametern des Presshärtens und dem daraus resultierenden Ziel-Parameter (Bauteilhärte) abzubilden.

### Data-Mining-Prozess:

![alt text](Prozess_Modellentwicklung_v2.png "Title")

### 0. Bibliotheken importieren

In [ ]:
# 0. Code-Block

# Importiere benötigte Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
%matplotlib inline

### 1. Daten erfassen

In [ ]:
# 1. Code-Block

# Erstelle eigene Zufallszahlen
my_seed = TODO

# Lade Datensatz und Ausgabe Zufallszahl
df = pd.read_excel("Daten_Presshärten.xlsx")
print("\nGewählte Zahl für Zufallszahlen: \t" + str(my_seed))

### 2. Daten erkunden

In [ ]:
# 2. Code-Block

# Letzten 5 Zeilen des Datensatzes
df.tail(5)

In [ ]:
# 3. Code-Block

# Streudiagramm-Matrix anzeigen
from pandas.plotting import scatter_matrix

scatter_matrix(df, alpha=0.2, figsize=(16, 16), diagonal="hist")
plt.show()

### 3. Daten vorbereiten - Aufteilen in Trainings- und Testdaten

Der Datensatz wird in Trainings- und Testdaten aufgeteilt. Die Trainingsdaten werden automatisch im Schritt GridSearch nochmals in Trainings- und Validationsdaten aufgeteilt.

In [ ]:
# 4. Code-Block

# Verhältnis Trainings- und Testdaten
test_size = TODO

# Teile Datensatz in Trainings- und Testdatensatz auf
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["Bauteilhärte"]),
    df["Bauteilhärte"],
    test_size=test_size,
    random_state=my_seed,
)

# Ausgabe Datensätze und Anzahl Datenpunkte
print("\nAnzahl Traingsdaten: \t\t" + str(len(y_train)) + " / " + str(len(df)))
print("Anzahl Testdaten: \t\t" + str(len(y_test)) + " / " + str(len(df)))

### 4.1 Modelle bilden - Mögliche Hyperparameter anzeigen

In [ ]:
# 5. Code-Block

# Importieren des Modells
from sklearn.neural_network import MLPRegressor 

# Ausgabe möglicher Hyperparameter
print("\nMögliche Hyperparameter (mit Standardeinstellung) eines Neuronalen Netzes:")
MLPRegressor().get_params()

Beschreibung der Hyperparameter:
http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

### 4.2 Modelle bilden - Optimale Hyperparameter mittels Gittersuche bestimmen

Um die optimalen Hyperparameter für das Neuronale Netz zu finden wird eine __Gittersuche__ durchgeführt:

TODO:
- Schreibe die zu variierenden Hyperparameter listenweise in ein Dictionary (6. Code-Block - Zeile 5-9)
- __Empfehlung:__ Verwende den solver lbfgs (nicht zwingend beste Möglichkeit): 'solver': ['lbfgs'], als random state deinen seed: random_state=my_seed
- Die Struktur eines Neuronalen Netzes (Anzahl an hidden layers sowie Neuronen je hidden layer) wird wie folgt angegeben:
    - hidden_layers_sizes = (3,) -> 1 hidden layer mit 3 Neuronen
    - hidden_layers_sizes = (3,2) -> 2 hidden layers mit 3 und 2 Neuronen
    - hidden_layers_sizes = (5,3,5) -> 3 hidden layers mit 5, 3 und 5 Neuronen
    - Zur Erinnerung: Die input layer hat soviele Neuronen wir Inputparameter, die output layer hat bei der Regression 1 Neuron

AUSGABE:
- Anzahl der Kombinationen, die Dauer der Gittersuche sowie das Modell mit den besten Hyperparametern

In [ ]:
# 6. Code-Block

# Werte für die Gittersuche/Hyperparameter
from sklearn.model_selection import GridSearchCV; import time; start_timer = time.monotonic()
hyper_parameters = [{'hidden_layer_sizes': [TODO], 
                     TODO,
                     TODO,
                     TODO,
                     'random_state': [my_seed],
                    }]

# Berechne Genauigkeit auf Validationsdaten für alle möglichen Kombinationen
mlp = MLPRegressor()
gridSearch = GridSearchCV(mlp, hyper_parameters, return_train_score=True, scoring='neg_mean_absolute_error', cv=5)
gridSearch = gridSearch.fit(X_train, y_train)
print('\nDie Gittersuche (' + str(len(pd.DataFrame(gridSearch.cv_results_))) + ' Kombinationen) hat ' + str("%.1f" % (time.monotonic()-start_timer)) + ' Sekunden gedauert, bestes Modell:')
gridSearch.best_estimator_

In [ ]:
# 7. Code-Block

# Ausgabe der Ergebnis-Tabelle der Gittersuche - besten 15 Modelle
pd.DataFrame(gridSearch.cv_results_).sort_values(
    "mean_test_score", ascending=False
).head(15)

### 4.3 Modell bilden - Vorzugs-Modell erstellen und trainieren

Das Modell mit den optimalen Hyperparametern wird erstell und trainiert. Danach wird der MAE auf den Trainingsdaten ausgegeben.

TODO:
- Schreibe den Code für den MAE des Modells auf den Trainingsdaten (Hilfestellung im Cheat-Sheet Modelle_testen) 

AUSGABE:
- MAE auf den Trainingsdaten

In [ ]:
# 8. Code-Block

# Erstellen des Modells - Verwenden des besten Modells aus der Gittersuche
best_mlp = gridSearch.best_estimator_

# Trainiere das Modell
best_mlp.fit(X_train, y_train)

# Ausgabe MAE auf Trainingsdaten
TODO

### 6.1 Modell testen & anwenden - Bewertung des Modells - MAE auf Testdaten

Um die Qualität/Güte des Modells zu bestimmen wird der MAE auf den Testdaten berechnet.

In [ ]:
# 9. Code-Block

# Berechne MAE auf Testdaten
TODO

__Vergleichswerte dieses Datensatzes Presshärten (MAE auf Testdaten):__
- Lineare Regression: 10.5
- Regressionsbaum: 10.2

### 6.2 Modell testen & anwenden - Bewertung des Modells - y über y_pred


TODO:
- Schreibe die Code-Zeilen für die Darstellung des MAE (Testdaten) von y über y_pred (Hilfestellung im Cheat-Sheet Modelle testen)

AUSGABE:
- Graph: MAE über y und y_pred

In [ ]:
# 10. Code-Block

# Ausgabe Graph: MAE über y und y_pred
y_now = y_test  # z.B. y_test
X_now = X_test  # z.B. X_test
model_now = best_mlp  # z.B. nn_model_best
min_test, max_test = y_now.min(), y_now.max()
fig = plt.figure(figsize=(9, 8))
plt.plot(y_now, np.squeeze(model_now.predict(X_now)), "o", alpha=0.4)
plt.plot([min_test, max_test], [min_test, max_test], "--", c=(0, 0, 0))
plt.xlabel("realer Wert")
plt.ylabel("Vorhersage")
plt.title("Modellgüte")
plt.xlabel("realer Wert")
plt.ylabel("Vorhersage")
plt.title("Modellgüte")
plt.show()

### 6.3 Modell testen & anwenden - Ausgabe der Gewichte

Die Gewichtsmatrizen enthalten die Parameter zur Berechnung der Werte der einzelnen Neuronen des NN (Berechnungsvorschrift von Schicht zu Schicht). Das untrainierte Netz besteht zu Beginn aus Gewichtsmatrizen mit zufälligen Werten. Im Laufe des Trainings werden die Werte der Gewichtsmatrizen solange angepasst (Backpropagation!) bis das NN einen möglichst geringen Fehler auf den Trainingsdaten aufzeigt und somit den Zusammenhang möglichst gut abbildet. Um ein NN abzuspeichern/zu rekonstruieren muss neben den Gewichtsmatrizen nur die Struktur sowie die Aktivierungsfunktionen gespeichert werden. 

Ausgabe der Gewichts-Matrizen des trainierten NN.

In [ ]:
# 11. Code-Block

# Ausgabe der Gewichts-Matrizen
best_mlp.coefs_

### Verständnisfragen:

1. Zeichne die Struktur deines besten NN.
2. Nenne mindestens 3 wichtige Hyperparameter deiner gridSearch.
3. Wie viele Neuronen muss die input layer für dieses Problem aufweisen?
4. Wie viele Neuronen muss die output layer für dieses Problem aufweisen?